# 问题1
### 描述
定义栈的数据结构，请在该类型中实现一个能够得到栈的最小元素的min函数。在栈中调用min.push及min.pop的时间复杂度都是O(1)。
### 分析
首先定义一个普通的栈，实现push和pop方法。要获得最小元素，可以从一个变量保存下当前栈中的最小元素，这样push的时候更新，在pop的时候也更新，但这在常熟时间内实现不了。从题目描述来看，min函数可以push和pop表示它也是一个栈，那么用另一个栈来保存当前栈中的所有最小元素即可，当原始栈push和pop的时候，要同步更新表示最小元素的栈。具体实现如下。

In [27]:
class Stack:
    def __init__(self):
        self.items = []
        
    def push(self, item):
        self.items.append(item)
    
    def pop(self):
        if (len(self.items) == 0):
            return None
        return self.items.pop()
    
    def show_all(self):
        print("all elements: %s" % str(self.items))
        
class MinStack(Stack):
    def __init__(self):
        self.min = Stack()
        super().__init__()
        
    def push(self, item):
        min_top = self.min.pop()
        if (min_top == None):
            self.min.push(item)
        else:
            if (item < min_top):
                self.min.push(min_top)
                self.min.push(item)
            else:
                self.min.push(min_top)
        super().push(item)
        
    def pop(self):
        top = super().pop()
        min_top = self.min.pop()
        if (top == None):
            return None
        if (top != min_top):
            self.min.push(min_top)
            
    def show_min(self):
        self.min.show_all()

In [28]:
stack = Stack()

stack.push(1)
stack.push(2)
stack.show_all()
stack.pop()
stack.show_all()

all elements: [1, 2]
all elements: [1]


In [32]:
min_stack = MinStack()
min_stack.push(6)
min_stack.push(3)
min_stack.push(4)
min_stack.show_all()
min_stack.show_min()
print(min_stack.min.pop())

all elements: [6, 3, 4]
all elements: [6, 3]
3


# 问题2

### 描述
根据两个文件的绝对路径，计算出后一个相对于前一个的相对路径。比如a=/qihoo/app/a/b/c/d/new.c，b=/qihoo/app/1/2/test.c，那么b相对于a的相对路径为../../../../1/2/test.c

### 分析
首先应该算出两个路径的最深公共目录，比如对于这两个路径为/qihoo/app。然后再算出被相对路径到此公共路径的深度，并算出目标路径去除公共路径的部分。最后前面算出的深度*`../`，再加上后面算出来的独立路径，即结果。

In [45]:
from functools import reduce

def relative_path(a, b):
    a_paths = a.split("/")
    b_paths = b.split("/")
    
    index = 0
    both_paths = []
    while (a_paths[index] == b_paths[index]):
        both_paths.append(a_paths[index])
        index += 1
    
    
    a_relative = a_paths[index:]
    b_relative = b_paths[index:]
    
    a_relative_depth = len(a_relative) - 1
    b_a = []
    while (a_relative_depth > 0):
        b_a.append("..")
        a_relative_depth -= 1
    
    b_a.extend(b_relative)
    
    return reduce(lambda x, y: x + "/" + y, b_a)

In [46]:
a = "/qihoo/app/a/b/c/d/new.c"
b = "/qihoo/app/1/2/test.c"
relative_path(a, b)

'../../../../1/2/test.c'

# 问题3

### 描述

给定一个字符串，判断它是不是合法的IP地址。

### 分析

这个问题有两种做法，其一是用正则表达式，其二是分割字符串然后判断。

In [11]:
import re

def test_ip(ip_strs):
    # 用正则表达式, 主要是匹配0～255这个范围
    re_0_199 = "[0,1]?\d{1,2}"
    re_200_249 = "2[0-4][0-9]"
    re_250_255 = "25[0-5]"
    re_200_255 = "2([0-4][0-9]|5[0-5])"
    re_0_255 = "(" + re_0_199 + "|" + re_200_255 + ")"
    re_ip = re_0_255 + "(\." + re_0_255 + ")" + "{3}"
        
    ipre = re.compile(re_ip)
    an = ipre.match(ip_strs)
    
    return an != None

ip_1 = "183.23.2.2"
ip_2 = "200.0.1903.2"
ip_3 = "123.w.32.3"
print(test_ip(ip_1))
print(test_ip(ip_2))
print(test_ip(ip_3))

True
False
False


# 问题4

### 描述

大数相加

### 分析

先将大数字按位分割，组成数组，然后从后向前按位相加。在相加的过程中，如果结果大于10，那么记住需要进位，而且还要注意最后一次相加的进位。

In [33]:
from functools import reduce

def big_add(a, b):
    
    def int_to_arr(n):
        # 将数字转换为按位保存的数组，并制逆序
        arr = list(str(n))
        arr.reverse()
        return [int(x) for x in arr]
    
    arr_a = int_to_arr(a)
    arr_b = int_to_arr(b)
    arr_an = []
    carry = False
    count = max(len(arr_a), len(arr_b))
    diff = len(arr_a) - len(arr_b)
    
    # 让数据位数对齐
    if (len(arr_a) > len(arr_b)):
        for i in range(diff):
            arr_b.append(0)
    elif (len(arr_b) > len(arr_a)):
        for i in range(-diff):
            arr_a.append(0)
    
    
    for i in range(count):
        an = arr_a[i] + arr_b[i]
        if (carry):
            an += 1
            
        if (an >= 10):
            carry = True
            an -= 10
        else:
            carry = False
            
        arr_an.append(an)
    
    if (carry):
        arr_an.append(1)
        
    arr_an.reverse()
    arr_str_an = [str(x) for x in arr_an]
    str_an = reduce(lambda x,y: str(x)+str(y), arr_str_an)
    
    return int(str_an)

print((big_add(149, 323)))
print(big_add(1, 3))
print(big_add(1000, 10))

472
4
1010


#  问题5
### 描述
将一个n元一维向量向左旋转i个位置。例如，当n=8且i=3时，向量abcdefgh旋转为defghabc。
### 分析
更加高效的算法是先将前i个元素置逆，然后将剩下的置逆，合并在一起，再做一次置逆。

In [51]:
from functools import reduce

def rotate(s, i):
    list_s = list(s)
    n = len(list_s)
    
    # if i > n
    i = i % n
    left = list_s[:i]
    left.reverse()
    right = list_s[i:]
    right.reverse()
    re_rl = left + right
    
    re_rl.reverse()
    
    return reduce(lambda x, y: x + y, re_rl)

rotate("abcdefg", 3)

'defgabc'

# 问题6
### 描述
在一个包含负数的向量中，求最大子向量和。
### 分析
常规的做法是枚举出n*(n-1)/2种可能性，然后算出最大值，这种方法低效。如果所有元素为负数，那么最大子向量和就为0，相反如果所有元素为正数，那么最大子向量和就为所有元素的和。所以，如果当前元素是负数，那么久要考虑是否将其纳入最终的子向量。很明显，如果【当前元素】+【之前子向量和】变成一个负数，那么【当前元素】不应该纳入最大子向量，而应该从下一个元素开始，重新计算。这样算到最后便可以得出最大子向量和。

In [54]:
def max_sum(vec):
    max_so_far = 0
    max_sf_here = 0
    for i in range(len(vec)):
        max_sf_here += vec[i]
        max_sf_here = max(max_sf_here, 0)
        max_so_far = max(max_so_far, max_sf_here)
    return max_so_far

max_sum([31, -41, 59, 26, -53, 58, 97, -93, -23, 84])

187